## Preprocessing

In [39]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from sklearn.inspection import permutation_importance
import numpy as np

# Import pandas and read the charity_data.csv from the provided cloud URL.

application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [40]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME"])
application_df.head()


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


What variable(s) are the target(s) for your model?
    - Is Successful
What variable(s) are the feature(s) for your model?
    - APPLICATION_TYPE, AFFILIATION, CLASSIFICATION, USE_CASE, ORGANIZATION, STATUS, INCOME_AMT, SPECIAL_CONSIDERATIONS, ASK_AMT        

In [41]:
# Determine the number of unique values in each column.
application_df.nunique()



APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [42]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
import matplotlib.pyplot as plt
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
print(application_type_counts)
# Determine which values to replace

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


The log scale was used to gain a better understanding of the distribution  of the less frequent application types and make a more informed decision about the cutoff value

In [43]:
# # Step 2: Visualize the distribution
# plt.figure(figsize=(10, 6))
# application_type_counts.plot(kind='bar')
# plt.title('Frequency Distribution of Application Types')
# plt.xlabel('Application Type')
# plt.ylabel('Frequency')
# plt.yscale('log')  # Use log scale for better visualization of outliers
# plt.show()


In [44]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
# Choose a cutoff value (example: 500)
cutoff_value = 530

# Create a list of application types to be replaced
application_types_to_replace = application_type_counts[application_type_counts < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: count, dtype: int64

In [45]:
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()

# plt.figure(figsize=(10, 6))
# application_type_counts .plot(kind='bar')
# plt.title('Frequency Distribution of Application Types')
# plt.xlabel('Application Type')
# plt.ylabel('Frequency')
# plt.yscale('log')  # Use log scale for better visualization of outliers
# plt.show()

In [46]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = application_df['CLASSIFICATION'].value_counts()
print(classification_counts)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64


In [47]:
# You may find it helpful to look at CLASSIFICATION value counts >1
print(classification_counts[classification_counts > 1])

# # Step 2: Visualize the distribution
# plt.figure(figsize=(10, 6))
# classification_counts.plot(kind='bar')  
# plt.title('Frequency Distribution of Classifications')
# plt.xlabel('Classification')
# plt.ylabel('Frequency')
# plt.yscale('log')  # Use log scale for better visualization of outliers
# plt.show()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64


In [48]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

classification_cutoff_value = 800

# Create a list of application types to be replaced
classifications_to_replace = classification_counts[classification_counts < classification_cutoff_value ].index.tolist()


for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [49]:
status_counts = application_df['STATUS'].value_counts()
print(status_counts)

STATUS
1    34294
0        5
Name: count, dtype: int64


In [50]:
# Convert categorical data to numeric with `pd.get_dummies`
categorical_columns = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION',  'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']

# Convert categorical data to numeric using pd.get_dummies
application_df = pd.get_dummies(application_df, columns=categorical_columns)

# Display the first few rows of the updated DataFrame
print(application_df.head())

   STATUS  ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_Other  \
0       1     5000              1                    True   
1       1   108590              1                   False   
2       1     5000              0                   False   
3       1     6692              1                   False   
4       1   142590              1                   False   

   APPLICATION_TYPE_T19  APPLICATION_TYPE_T3  APPLICATION_TYPE_T4  \
0                 False                False                False   
1                 False                 True                False   
2                 False                False                False   
3                 False                 True                False   
4                 False                 True                False   

   APPLICATION_TYPE_T5  APPLICATION_TYPE_T6  APPLICATION_TYPE_T7  ...  \
0                False                False                False  ...   
1                False                False                False  ...   

In [51]:
# Split the data into features and target
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop(columns='IS_SUCCESSFUL')

# Capture feature names
feature_names = X.columns

# Convert to numpy array for scaling
X = X.values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)



In [52]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [53]:
# Function to create the model
def create_model(input_dim):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=80, activation='relu', input_dim=input_dim))
    model.add(tf.keras.layers.Dense(units=30, activation='relu'))
    model.add(tf.keras.layers.Dense(units=15, activation='relu'))  # Third hidden layer
    model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [54]:
# Function to train the model
def train_model(model, X_train, y_train, X_test, y_test, epochs=100):
    # Train the model
    fit_model = model.fit(X_train, y_train, epochs=epochs, batch_size=32, validation_split=0.2)
    
    # Evaluate the model using the test data
    model_loss, model_accuracy = model.evaluate(X_test, y_test, verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
    
    return model, fit_model

In [55]:
# Create the model
nn = create_model(input_dim=X_train_scaled.shape[1])

# Train the model with increased epochs
nn.fit(X_train_scaled, y_train, epochs=150, batch_size=32, validation_split=0.2)

Epoch 1/150
644/644 [==============================] - 1s 1ms/step - loss: 0.5726 - accuracy: 0.7163 - val_loss: 0.5500 - val_accuracy: 0.7353
Epoch 2/150
644/644 [==============================] - 1s 1ms/step - loss: 0.5563 - accuracy: 0.7272 - val_loss: 0.5521 - val_accuracy: 0.7339
Epoch 3/150
644/644 [==============================] - 1s 1ms/step - loss: 0.5532 - accuracy: 0.7296 - val_loss: 0.5487 - val_accuracy: 0.7388
Epoch 4/150
644/644 [==============================] - 1s 1ms/step - loss: 0.5520 - accuracy: 0.7294 - val_loss: 0.5497 - val_accuracy: 0.7374
Epoch 5/150
644/644 [==============================] - 1s 1ms/step - loss: 0.5509 - accuracy: 0.7284 - val_loss: 0.5478 - val_accuracy: 0.7374
Epoch 6/150
644/644 [==============================] - 1s 1ms/step - loss: 0.5499 - accuracy: 0.7317 - val_loss: 0.5459 - val_accuracy: 0.7347
Epoch 7/150
644/644 [==============================] - 1s 1ms/step - loss: 0.5485 - accuracy: 0.7321 - val_loss: 0.5464 - val_accuracy: 0.7374

In [56]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5742 - accuracy: 0.7236 - 214ms/epoch - 800us/step
Loss: 0.57424396276474, Accuracy: 0.7236151695251465


In [57]:
# Custom function to calculate permutation importance
def custom_permutation_importance(model, X_test, y_test, feature_names, n_repeats=10):
    baseline_accuracy = model.evaluate(X_test, y_test, verbose=0)[1]
    importances = np.zeros(X_test.shape[1])
    for i in range(X_test.shape[1]):
        scores = np.zeros(n_repeats)
        for n in range(n_repeats):
            X_test_permuted = X_test.copy()
            np.random.shuffle(X_test_permuted[:, i])
            permuted_accuracy = model.evaluate(X_test_permuted, y_test, verbose=0)[1]
            scores[n] = baseline_accuracy - permuted_accuracy
        importances[i] = np.mean(scores)
    return importances


In [59]:
import csv

# Calculate permutation importance
importances = custom_permutation_importance(nn, X_test_scaled, y_test, feature_names)

# Save permutation importance to a CSV file
with open('permutation_importance.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Feature', 'Importance'])
    for i in np.argsort(importances)[::-1]:
        writer.writerow([feature_names[i], f"{importances[i]:.4f}"])

# Display permutation importance
for i in np.argsort(importances)[::-1]:
    print(f"{feature_names[i]}: {importances[i]:.4f}")

AFFILIATION_CompanySponsored: 0.0707
AFFILIATION_Independent: 0.0459
APPLICATION_TYPE_T4: 0.0162
ORGANIZATION_Association: 0.0156
APPLICATION_TYPE_T19: 0.0153
APPLICATION_TYPE_Other: 0.0140
APPLICATION_TYPE_T6: 0.0101
ORGANIZATION_Trust: 0.0092
CLASSIFICATION_Other: 0.0079
CLASSIFICATION_C3000: 0.0072
APPLICATION_TYPE_T5: 0.0063
USE_CASE_ProductDev: 0.0060
APPLICATION_TYPE_T8: 0.0059
INCOME_AMT_10M-50M: 0.0038
CLASSIFICATION_C1200: 0.0032
APPLICATION_TYPE_T7: 0.0030
APPLICATION_TYPE_T3: 0.0030
INCOME_AMT_100000-499999: 0.0025
INCOME_AMT_50M+: 0.0022
INCOME_AMT_1M-5M: 0.0018
USE_CASE_Preservation: 0.0016
INCOME_AMT_5M-10M: 0.0014
INCOME_AMT_1-9999: 0.0012
ORGANIZATION_Co-operative: 0.0010
CLASSIFICATION_C2100: 0.0009
CLASSIFICATION_C2000: 0.0009
CLASSIFICATION_C1000: 0.0008
USE_CASE_Heathcare: 0.0006
INCOME_AMT_10000-24999: 0.0005
ASK_AMT: 0.0005
USE_CASE_CommunityServ: 0.0005
ORGANIZATION_Corporation: 0.0003
AFFILIATION_National: 0.0002
USE_CASE_Other: 0.0001
AFFILIATION_Other: 0.0000


In [ ]:
# Save the optimized model to an HDF5 file
nn.save('AlphabetSoupCharity_Optimized2stAttempt.h5')

c:\Users\jimco\anaconda3\envs\ml_env\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
